# Zero-Shot Prompting

Zero-shot prompting refers to a technique in prompt engineering where you provide a model with a task without any prior examples. The model is expected to understand and generate a response or complete the task purely based on the given instruction.

In other words, the model is given "zero" prior training examples or demonstrations in the prompt and relies on its pre-trained knowledge to infer what is needed.

## References:
* [Wei et al. (2022)](https://arxiv.org/pdf/2109.01652.pdf): demonstrate how instruction tuning improves zero-shot learning 

## Running this code on MyBind.org

Note: remember that you will need to **adjust CONFIG** with **proper URL and API_KEY**!

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/GenILab-FAU/prompt-eng/HEAD?urlpath=%2Fdoc%2Ftree%2Fprompt-eng%2Fzero_shot.ipynb)



In [ ]:
############################################################
## Install required packages
############################################################

!pip install requests

# Restart the kernel to ensure the installation takes effect
import IPython
IPython.Application.instance().kernel.do_shutdown(True)



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


{'status': 'ok', 'restart': True}

: 

In [2]:
############################################################
## ZERO-SHOT PROMPT: PROJECT OVERVIEW
############################################################

from _pipeline import create_payload, model_req

# 1) Define the prompt (no examples, purely zero-shot)
ZERO_SHOT_PROMPT = """You are an AI that generates project reports.
Please create the "Project Overview" section covering:
- Title
- Goal
- Problem Statement
- Key Objectives
- Scope
Format it in clear paragraphs, and use Markdown headings.
"""

# 2) Prepare the payload (choose your model & parameters)
payload = create_payload(
    target="open-webui",    # or whichever backend you're using
    model="phi4:latest",    # example model; adjust as needed
    prompt=ZERO_SHOT_PROMPT,
    temperature=1.0,        # higher temp => more creative text
    num_ctx=100,
    num_predict=200         # enough tokens for a full overview
)

# 3) Send request and capture the response
time_taken, response = model_req(payload=payload)

# 4) Display the raw output and timing
print("===== ZERO-SHOT OUTPUT =====")
print(response)
if time_taken:
    print(f"\nTime taken: {time_taken}s")


ModuleNotFoundError: No module named 'requests'

---

## How to improve it?

* **Use Clear and Concise Instructions**: Be specific about the task and desired format.
    * Bad Prompt: “Summarize this.”
    * Good Prompt: “Summarize this paragraph in one sentence.”
* **Add Context**: Providing background can help the model interpret ambiguous prompts better.
* **Specify Output Format**: If a particular structure is needed, describe it in the instruction.